In [1]:
import pandas as pd
import numpy as np
from tabulate import tabulate

In [2]:
# Relation class representing a database table
class Relation:
    def __init__(self, colnames, data, prim_key):
        """
        Initializes the Relation class with table attributes:
        - colnames: list of column names
        - data: list of data rows
        - prim_key: list representing the primary key columns
        """
        self.name = ""  # Name of the table (relation)
        self.data = data  # Data rows of the table
        self.prim_key = prim_key  # List of columns that form the primary key
        self.key_foreign = []  # List to store foreign keys
        self.colnames = colnames  # List of column names for the table
        self.dependency = []  # List to store dependencies (functional or multi-valued)

    def add_table_name(self, name):
        """Assigns a name to the table (relation)."""
        self.name = name

    def foreignkey_addition(self, key_foreign):
        """
        Adds a foreign key to the relation.
        - key_foreign: Tuple containing (foreign key column, referenced table, referenced column)
        """
        self.key_foreign.append(key_foreign)

    def dependency_adding(self, dependency):
        """
        Adds a dependency to the relation.
        - dependency: Instance of a Dependency object
        """
        self.dependency.append(dependency)

    def show(self):
        """Displays the table name, data, primary key, and functional dependencies."""
        print(self.name)
        # Pretty-print the data using column names as headers
        print(tabulate(self.data, headers=self.colnames, tablefmt='pretty'))
        print("Primary Key:", self.prim_key)
        
        # Display functional and multi-valued dependencies
        print("Functional dependencies for tables are:")
        for dep in self.dependency:
            # Determine dependency type: Functional Dependency (FD) or Multi-Valued Dependency (MVD)
            type_of_dependency = "FD" if dep.dependency_type == "FD" else "MVD"
            print(f"  LHS: {dep.lhs} -> RHS: {dep.rhs} (Type: {type_of_dependency})")

    def table_formation(self):
        """
        Generates the SQL CREATE TABLE statement for the relation, including primary key and foreign key constraints.
        """
        if not self.name:
            raise ValueError("Relation is not assigned a name, please use add_table_name method")

        # Start building the SQL statement
        tabule = f"CREATE TABLE {self.name} (\n"
        col_definations = []

        # Define each column as VARCHAR(255)
        for a in self.colnames:
            col_definations.append(f"    {a} VARCHAR(255)")

        # Define the primary key constraint
        prim_key_str = f"    PRIMARY KEY ({', '.join(self.prim_key)})"
        col_definations.append(prim_key_str)

        # Add foreign key constraints if any exist
        for col, reference_tab, reference_col in self.key_foreign:
            fk_str = f"    FOREIGN KEY ({col}) REFERENCES {reference_tab}({reference_col})"
            col_definations.append(fk_str)

        # Combine all column definitions into the final SQL statement
        tabule += ",\n".join(col_definations)
        tabule += "\n);"

        return tabule


In [3]:
# Dependency class represents a dependency (functional or multi-valued) between columns
class Dependency:
    def __init__(self, depen_string, key_candid):
        """
        Initializes the Dependency class with:
        - depen_string: String format of the dependency (e.g., "A -> B" or "A -->> B").
        - key_candid: List of candidate keys to check partial dependency.
        """
        self.depen_string = depen_string.strip()  # Clean dependency string
        self.key_candid = key_candid  # Candidate keys of the relation
        self.lhs = []  # Left-hand side of the dependency
        self.rhs = []  # Right-hand side of the dependency
        self.dependency_type = ""  # Type of dependency: "FD" or "MVD"
        
        # Parse the dependency string to initialize lhs, rhs, and dependency type
        self.depen_parsing()

    def depen_parsing(self):
        """
        Parses the dependency string to determine dependency type and separate LHS and RHS.
        """
        # Check if it is a Multi-Valued Dependency (MVD)
        if '-->>' in self.depen_string:
            self.lhs, rhs_part = self.depen_string.split('-->>')
            self.dependency_type = 'MVD'
        # Check if it is a Functional Dependency (FD)
        elif '->' in self.depen_string:
            self.lhs, rhs_part = self.depen_string.split('->')
            self.dependency_type = 'FD'
        else:
            # Raise an error if the format is invalid
            raise ValueError("Dependency format is Invalid")

        # Strip whitespace and split LHS and RHS into lists
        self.lhs = [x.strip() for x in self.lhs.split(',')]
        self.rhs = [x.strip() for x in rhs_part.split(',')]

    def check_depend_paren(self):
        """
        Checks if the dependency is a partial dependency based on candidate keys.
        Returns True if it is a partial dependency, False otherwise.
        """
        # A dependency is partial if LHS is a proper subset of any candidate key
        for k in self.key_candid:
            if set(self.lhs).issubset(set(k)) and set(self.lhs) != set(k):
                return True
        return False

    def __str__(self):
        """
        String representation of the dependency showing LHS, RHS, type, and partial dependency status.
        """
        return f"LHS: {self.lhs}, RHS: {self.rhs}, Type: {self.dependency_type}, Partial: {self.check_depend_paren()}"


In [4]:
def is_relation_1nf(rltn):
    """
    Checks if a relation is in 1NF (First Normal Form).
    - rltn: Relation object
    Returns a tuple (if_in_1nf_relation, non_1nf_columns) where:
    - if_in_1nf_relation: True if relation is in 1NF, False otherwise
    - non_1nf_columns: List of columns that violate 1NF (i.e., have multi-valued attributes)
    """
    non_1nf_columns = set()  # Set to collect columns that violate 1NF
    
    # Iterate over each row in the data
    for records in rltn.data:
        for idx, value in enumerate(records):
            # Check if the value is a multi-valued attribute (comma-separated string)
            if isinstance(value, str) and ',' in value:
                non_1nf_columns.add(rltn.colnames[idx])  # Add violating column name
    
    # Check if there are no columns with multi-valued attributes
    if_in_1nf_relation = len(non_1nf_columns) == 0
    return if_in_1nf_relation, list(non_1nf_columns)


def normalize_1nf_relation(rltn, attr_multi_val):
    """
    Normalizes a relation to 1NF by removing multi-valued attributes.
    - rltn: Relation object
    - attr_multi_val: List of attributes (columns) that have multi-valued entries
    Returns a dictionary containing:
    - Base table without multi-valued attributes
    - New tables for each multi-valued attribute
    """
    # Get the indices of primary key columns
    pk_idx = [rltn.colnames.index(pk) for pk in rltn.prim_key]
    
    # Get the indices of multi-valued attributes
    mv_idx = {attribute: rltn.colnames.index(attribute) for attribute in attr_multi_val if attribute in rltn.colnames}
    
    # Create the base table column names by excluding multi-valued attributes
    col_base_names = [hi for hi in rltn.colnames if hi not in attr_multi_val]
    
    # Populate base table data by excluding multi-valued attribute values in each row
    base_table_data = [[records[i] for i, h in enumerate(rltn.colnames) if h not in attr_multi_val] for records in rltn.data]
    
    # Create the base relation without multi-valued attributes
    base_table = Relation(col_base_names, base_table_data, rltn.prim_key)
    base_table.add_table_name("BaseRelation")  # Name the base table
    
    new_tableso = {}  # Dictionary to store new tables created for each multi-valued attribute

    # Iterate over each multi-valued attribute and its index
    for attribute, idx in mv_idx.items():
        # Define new table columns: primary key + multi-valued attribute column
        new_tables_colnames = rltn.prim_key + [attribute]
        new_tables_data = []

        # Split multi-valued entries and create new rows for the new table
        for records in rltn.data:
            values = str(records[idx]).split(',')  # Split multi-valued attribute by commas
            for value in values:
                # Combine primary key values with each individual value of the multi-valued attribute
                new_records = [records[i] for i in pk_idx] + [value.strip()]
                new_tables_data.append(new_records)

        # Create new relation table for each multi-valued attribute
        new_tables = Relation(new_tables_colnames, new_tables_data, rltn.prim_key + [attribute])
        new_tables.add_table_name(attribute)  # Name the new table after the attribute

        # Add foreign key constraint referencing the base table for each primary key
        for pk in rltn.prim_key:
            new_tables.foreignkey_addition((pk, base_table.name, pk))

        # Store the new table in the dictionary
        new_tableso[attribute] = new_tables

    # Return a dictionary with the base table and all newly created tables for multi-valued attributes
    return {
        "BaseRelation": base_table,
        **new_tableso
    }


In [5]:
def if_relation_in_2nf(relation):
    """
    Checks if a relation is in 2NF (Second Normal Form).
    - relation: Relation object
    Returns True if the relation is in 2NF, False otherwise.
    """
    # Check if the relation is in 1NF; if not, it can't satisfy 2NF
    if not is_relation_1nf(relation):
        print(f"Relation '{relation.name}' is not in 1NF, so it does not satisfy 2NF.")
        return False

    # Extract the primary key and non-prime attributes (attributes not in the primary key)
    prim_key = relation.prim_key
    attr_non_prime = [a for a in relation.colnames if a not in prim_key]

    # If the primary key is a single attribute, it can't have partial dependencies
    if len(prim_key) == 1:
        return True

    # Check dependencies for partial dependencies
    for dependency in relation.dependency:
        lhs_set = set(dependency.lhs)
        rhs_set = set(dependency.rhs)

        # A partial dependency exists if LHS is a proper subset of the primary key
        if lhs_set.issubset(set(prim_key)) and lhs_set != set(prim_key):
            if any(a in attr_non_prime for a in rhs_set):
                print(f"Relation '{relation.name}' contains partial dependency: {dependency.lhs} -> {dependency.rhs}")
                return False

    return True


def check_all_relations_2nf(rltn):
    """
    Checks if all relations in a dictionary satisfy 2NF.
    - rltn: Dictionary of Relation objects
    Returns True if all relations are in 2NF, False otherwise.
    """
    for name, rel in rltn.items():
        if if_relation_in_2nf(rel):
            continue
        else:
            return False
    return True


def decompose_to_2nf(relations):
    """
    Decomposes relations to 2NF if they contain partial dependencies.
    - relations: Dictionary of Relation objects
    Returns a new dictionary of relations in 2NF.
    """
    new_tableso = {}  # Dictionary to store new normalized relations

    for table_name, relation in relations.items():
        prim_key = set(relation.prim_key)
        dependency_partial = []  # Stores partial dependencies
        dependency_fullfunctional = []  # Stores full functional dependencies
        dependency_trans = []  # Stores transitive dependencies
        dependency_mvd = []  # Stores multi-valued dependencies

        # Separate dependencies into partial, full functional, transitive, and multi-valued
        for d in relation.dependency:
            lhs_set = set(d.lhs)
            # Identify partial dependencies
            if d.dependency_type == 'FD' and lhs_set.issubset(prim_key) and lhs_set != prim_key:
                if d.check_depend_paren():
                    dependency_partial.append(d)
                else:
                    dependency_fullfunctional.append(d)
            elif not lhs_set.issubset(prim_key):
                # Dependencies where LHS is not a subset of the primary key might be transitive
                dependency_trans.append(d)
            elif d.dependency_type == 'MVD':
                dependency_mvd.append(d)

        # If partial dependencies exist, decompose the relation
        if dependency_partial:
            # Create new relations for each partial dependency
            for depend_par in dependency_partial:
                new_tab_name = f"{table_name}_Partial_{len(new_tableso) + 1}"
                new_record_names = list(set(depend_par.lhs + depend_par.rhs))
                new_tab_d = []  # Data for the new partial dependency relation
                new_prim_key = depend_par.lhs  # Use LHS as the new primary key

                # Check if any of the RHS attributes are part of the primary key
                prim_key_in_rhs = any(attr in prim_key for attr in depend_par.rhs)
                if prim_key_in_rhs:
                    # Update primary key if RHS contains primary key attributes
                    new_prim_key = list(set(depend_par.lhs + depend_par.rhs))
                else:
                    new_prim_key = depend_par.lhs

                record_exist = set()  # Set to track unique records
                # Populate data for the new table
                for row in relation.data:
                    new_records = [row[relation.colnames.index(attr)] for attr in new_record_names]
                    if tuple(new_records) not in record_exist:
                        new_tab_d.append(new_records)
                        record_exist.add(tuple(new_records))

                # Create a new relation for the partial dependency
                new_tables = Relation(new_record_names, new_tab_d, new_prim_key)
                new_tables.add_table_name(new_tab_name)

                # Add dependency to the new table
                depen_new = Dependency(f"{','.join(depend_par.lhs)} -> {','.join(depend_par.rhs)}", [new_prim_key])
                new_tables.dependency_adding(depen_new)

                # Add foreign key constraint to reference the original relation
                for pk in relation.prim_key:
                    if pk in new_record_names and pk not in new_prim_key:
                        new_tables.foreignkey_addition((pk, table_name, pk))

                # Add transitive dependencies to the new table if they apply
                for depen_trans in dependency_trans:
                    if all(a in new_record_names for a in depen_trans.lhs + depen_trans.rhs):
                        new_tables.dependency_adding(depen_trans)

                new_tableso[new_tab_name] = new_tables

            # Update the original relation with remaining attributes after decomposition
            fields_remain = list(set(relation.colnames) - set(sum([dep.rhs for dep in dependency_partial], [])))
            data_remain = []

            # Update the primary key for the remaining relation
            prim_key_remain = list(set(prim_key).intersection(fields_remain))
            if not prim_key_remain:
                prim_key_remain = relation.prim_key

            # Update the remaining dependencies, including transitive ones
            depen_remain = []
            for d in dependency_fullfunctional + dependency_trans + dependency_mvd:
                if all(attr in fields_remain for attr in d.lhs + d.rhs):
                    d.key_candid = [prim_key_remain]
                    depen_remain.append(d)

            record_exist = set()  # Set to track unique records
            for r in relation.data:
                new_records = [r[relation.colnames.index(a)] for a in fields_remain]
                if tuple(new_records) not in record_exist:
                    data_remain.append(new_records)
                    record_exist.add(tuple(new_records))

            # Create the updated original relation
            new_tables = Relation(fields_remain, data_remain, prim_key_remain)
            new_tables.add_table_name(table_name)
            new_tables.dependency = depen_remain

            # Add foreign keys to link with partial dependency tables
            for rltn_par_name, par_rel in new_tableso.items():
                for pk in par_rel.prim_key:
                    if pk in fields_remain and pk not in prim_key_remain:
                        new_tables.foreignkey_addition((pk, rltn_par_name, pk))

            new_tableso[table_name] = new_tables

        else:
            # If no partial dependencies exist, retain the original relation
            new_tableso[table_name] = relation

    return new_tableso


In [6]:
def if_relation_in_3nf(relation):
    """
    Checks if a relation is in 3NF (Third Normal Form).
    - relation: Relation object
    Returns True if the relation is in 3NF, False otherwise.
    """
    # First, check if the relation is in 2NF
    if not if_relation_in_2nf(relation):
        print(f"Relation '{relation.name}' is not in 2NF, so the relation won't be in 3NF.")
        return False

    prim_key = set(relation.prim_key)
    key_candid = [set(relation.prim_key)]  # Assuming the primary key is the candidate key
    attr_non_prime = [a for a in relation.colnames if a not in prim_key]

    # Check for transitive dependency violations for 3NF
    for d in relation.dependency:
        lhs_set = set(d.lhs)
        rhs_set = set(d.rhs)

        # Violation if LHS is not a superkey and RHS has non-prime attributes
        if not lhs_set.issuperset(prim_key) and not any(a in prim_key for a in rhs_set):
            if any(a in attr_non_prime for a in rhs_set):
                print(f"Relation '{relation.name}' has a transitive dependency: {d.lhs} -> {d.rhs}")
                return False

    # If no transitive dependencies are found, it's in 3NF
    return True

def relations_in_3nf(relations):
    """
    Checks if all relations in a dictionary are in 3NF.
    - relations: Dictionary of Relation objects
    Returns True if all relations are in 3NF, False otherwise.
    """
    f = True
    for n, rel in relations.items():
        if if_relation_in_3nf(rel):
            print(f"Relation '{n}' in 3NF.")
            continue
        else:
            print(f"Relation '{n}' not in 3NF.")
            f = False
    return f

def tracking_primary_keys(relations):
    """
    Tracks and maps primary keys to their respective relation tables.
    - relations: Dictionary of Relation objects
    Returns a dictionary mapping each primary key to its table name.
    """
    prim_key_map = {}
    for table_name, relation in relations.items():
        for pk in relation.prim_key:
            prim_key_map[pk] = table_name
    return prim_key_map

def is_key_superkey(relation, lhs_set):
    """
    Checks if a given set of attributes is a superkey.
    - relation: Relation object
    - lhs_set: Set of attributes to check
    Returns True if lhs_set is a superkey, False otherwise.
    """
    prim_key_set = set(relation.prim_key)
    return lhs_set.issuperset(prim_key_set)

def decompose_to_3nf(relations):
    """
    Decomposes relations to 3NF by eliminating transitive dependencies.
    - relations: Dictionary of Relation objects
    Returns a new dictionary of relations in 3NF.
    """
    prim_key_map = tracking_primary_keys(relations)
    new_tableso = {}  # To store new 3NF relations

    for table_name, relation in relations.items():
        prim_key = set(relation.prim_key)
        dependency_trans = []  # Transitive dependencies
        non_dependency_trans = []  # Non-transitive dependencies

        # Separate transitive and non-transitive dependencies
        for d in relation.dependency:
            lhs_set = set(d.lhs)
            rhs_set = set(d.rhs)

            if not lhs_set.issuperset(prim_key) and not any(attr in prim_key for attr in rhs_set):
                dependency_trans.append(d)
            else:
                non_dependency_trans.append(d)

        # For each transitive dependency, create a new relation
        for depen_trans in dependency_trans:
            new_tab_name = f"{table_name}_Transitive_{len(new_tableso) + 1}"
            new_record_names = list(set(depen_trans.lhs + depen_trans.rhs))
            new_tab_d = []
            new_prim_key = depen_trans.lhs  # Set LHS as the new primary key

            # Populate the new table with records
            record_exist = set()
            for row in relation.data:
                new_records = [row[relation.colnames.index(a)] for a in new_record_names]
                if tuple(new_records) not in record_exist:
                    new_tab_d.append(new_records)
                    record_exist.add(tuple(new_records))

            # Create the new relation with transitive dependency
            new_tables = Relation(new_record_names, new_tab_d, new_prim_key)
            new_tables.add_table_name(new_tab_name)

            # Add dependency to the new table
            depen_new = Dependency(f"{','.join(depen_trans.lhs)} -> {','.join(depen_trans.rhs)}", [new_prim_key])
            new_tables.dependency_adding(depen_new)

            # Add foreign key constraints
            for a in new_record_names:
                if a in prim_key_map and a not in new_prim_key:
                    src_tab = prim_key_map[a]
                    new_tables.foreignkey_addition((a, src_tab, a))

            new_tableso[new_tab_name] = new_tables

        # If transitive dependencies exist, update the original relation
        if dependency_trans:
            fields_remain = list(set(relation.colnames) - set(sum([dep.rhs for dep in dependency_trans], [])))
            data_remain = []

            # Update the primary key for the remaining relation
            prim_key_remain = list(set(prim_key).intersection(fields_remain))
            if not prim_key_remain:
                prim_key_remain = relation.prim_key

            # Update dependencies, excluding transitive ones
            depen_remain = []
            for d in non_dependency_trans:
                if all(attr in fields_remain for attr in d.lhs + d.rhs):
                    d.key_candid = [prim_key_remain]
                    depen_remain.append(d)

            # Populate the remaining data
            record_exist = set()
            for r in relation.data:
                new_records = [r[relation.colnames.index(a)] for a in fields_remain]
                if tuple(new_records) not in record_exist:
                    data_remain.append(new_records)
                    record_exist.add(tuple(new_records))

            # Create the updated original relation with remaining attributes
            new_tables = Relation(fields_remain, data_remain, prim_key_remain)
            new_tables.add_table_name(table_name)
            new_tables.dependency = depen_remain

            # Add foreign keys linking with transitive tables
            for attr in fields_remain:
                if attr in prim_key_map and attr not in prim_key_remain:
                    src_tab = prim_key_map[attr]
                    new_tables.foreignkey_addition((attr, src_tab, attr))

            new_tableso[table_name] = new_tables

        else:
            # If no transitive dependencies exist, retain the original relation
            new_tableso[table_name] = relation

    return new_tableso


In [7]:
def if_relation_in_bcnf(relation):
    """
    Checks if a relation is in BCNF (Boyce-Codd Normal Form).
    - relation: Relation object
    Returns True if the relation is in BCNF, False otherwise.
    """
    # First, verify if the relation is in 3NF, a requirement for BCNF
    if not if_relation_in_3nf(relation):
        print(f"Relation '{relation.name}' is not in 3NF, so it does not satisfy BCNF.")
        return False

    # Check each dependency to see if it violates BCNF conditions
    for d in relation.dependency:
        lhs_set = set(d.lhs)
        # If LHS is not a superkey and it's not a multi-valued dependency (MVD), it violates BCNF
        if not is_key_superkey(relation, lhs_set) and not d.dependency_type == "MVD":
            print(f"Relation '{relation.name}' violates BCNF: {d.lhs} -> {d.rhs}")
            return False

    return True

def relation_is_in_bcnf(relations):
    """
    Checks if all relations in a dictionary are in BCNF.
    - relations: Dictionary of Relation objects
    Returns True if all relations are in BCNF, False otherwise.
    """
    f = True
    for n, r in relations.items():
        if if_relation_in_bcnf(r):
            # Uncomment below line to print BCNF status for each relation
            # print(f"Relation '{n}' is in BCNF.")
            continue
        else:
            # Uncomment below line to print BCNF violation status
            # print(f"Relation '{n}' is not in BCNF.")
            f = False
    return f

def remove_duplicate_relations(relations):
    """
    Removes duplicate relations from a dictionary by checking primary keys and column names.
    - relations: Dictionary of Relation objects
    Returns a dictionary of unique relations.
    """
    uni_rel = {}
    known_prim_key = {}

    # Iterate through each relation
    for n, rel in relations.items():
        # Create a tuple with sorted primary key and sorted column names as unique identifiers
        prim_key_tuple = (tuple(sorted(rel.prim_key)), tuple(sorted(rel.colnames)))

        # If a unique tuple for primary key and columns is new, add to unique relations
        if prim_key_tuple not in known_prim_key:
            known_prim_key[prim_key_tuple] = n
            uni_rel[n] = rel
        else:
            # Uncomment below line to print duplicate relation information
            # print(f"Skipping duplicate relation: {n}, keeping {known_prim_key[prim_key_tuple]}")
            pass

    return uni_rel

def bcnf_decomposition(relations):
    """
    Decomposes relations to BCNF by iteratively handling BCNF violations.
    - relations: Dictionary of Relation objects
    Returns a dictionary of relations decomposed to BCNF.
    """
    # Check if all relations are already in BCNF
    if relation_is_in_bcnf(relations):
        print("All relations are in BCNF.")
        return relations  

    # Initialize structures for decomposition
    prim_key_map = tracking_primary_keys(relations)
    new_tableso = {}  # To store the decomposed BCNF relations
    rel_que = list(relations.items())  # Queue for processing relations

    # Iterate through each relation in the queue
    while rel_que:
        table_name, relation = rel_que.pop(0)
        prim_key = set(relation.prim_key)
        violate_bcnf = []

        # Identify dependencies that violate BCNF
        for dep in relation.dependency:
            lhs_set = set(dep.lhs)
            if not is_key_superkey(relation, lhs_set):
                violate_bcnf.append(dep)

        # If there are BCNF violations, decompose the relation
        if violate_bcnf:
            # Pick the first violating dependency
            violate_dependency = violate_bcnf[0]
            lhs_attri = violate_dependency.lhs
            rhs_attri = violate_dependency.rhs

            # Create a new relation for the violating dependency
            new_tab_name = f"{table_name}_BCNF_{len(new_tableso) + 1}"
            new_record_names = list(set(lhs_attri + rhs_attri))
            new_tab_d = []
            new_prim_key = lhs_attri  # Set LHS as the new primary key

            # Populate the new relation with records based on the violating dependency
            record_exist = set()
            for r in relation.data:
                new_records = [r[relation.colnames.index(attr)] for attr in new_record_names]
                if tuple(new_records) not in record_exist:
                    new_tab_d.append(new_records)
                    record_exist.add(tuple(new_records))

            # Initialize the new relation with its attributes and dependency
            new_tables = Relation(new_record_names, new_tab_d, new_prim_key)
            new_tables.add_table_name(new_tab_name)
            depen_new = Dependency(f"{','.join(lhs_attri)} -> {','.join(rhs_attri)}", [new_prim_key])
            new_tables.dependency_adding(depen_new)

            # Add foreign key relationships
            for a in new_record_names:
                if a in prim_key_map and a not in new_prim_key:
                    src_tab = prim_key_map[a]
                    new_tables.foreignkey_addition((a, src_tab, a))

            new_tableso[new_tab_name] = new_tables

            # Create the remaining relation with fields after removing violating attributes
            fields_remain = list(set(relation.colnames) - set(rhs_attri) | set(lhs_attri))
            data_remain = []
            prim_key_remain = relation.prim_key

            # Populate the remaining data after the decomposition
            record_exist = set()
            for r in relation.data:
                new_records = [r[relation.colnames.index(a)] for a in fields_remain]
                if tuple(new_records) not in record_exist:
                    data_remain.append(new_records)
                    record_exist.add(tuple(new_records))

            # Initialize the remaining relation with updated attributes
            update_tab = Relation(fields_remain, data_remain, prim_key_remain)
            update_tab.add_table_name(table_name)

            # Re-add the updated relation to the queue for further checking
            rel_que.append((table_name, update_tab))

        else:
            # If no BCNF violations, retain the original relation
            new_tableso[table_name] = relation
            print(f"Relation '{table_name}' is already in BCNF.")

    return new_tableso


In [8]:
def if_relation_in_4nf(relation):
    # Check if the relation is in BCNF (Boyce-Codd Normal Form)
    if not if_relation_in_bcnf(relation):
        # If not in BCNF, it can't be in 4NF
        return False

    # Iterate through all functional dependencies in the relation
    for d in relation.dependency:
        lhs_set = set(d.lhs)  # Left-hand side of the dependency
        # Check for Multivalued Dependencies (MVDs)
        if d.dependency_type == "MVD" and not is_key_superkey(relation, lhs_set):
            # If MVD exists and the LHS is not a superkey, it's a violation of 4NF
            return False
    return True  # Relation is in 4NF

def verify_relations_in_4nf(relations):
    # Verify all relations in the provided dictionary
    f = True
    for n, rel in relations.items():
        # Check each relation for 4NF
        if if_relation_in_4nf(rel):
            # Optionally print confirmation (commented out)
            continue
        else:
            # If any relation is not in 4NF, mark the result as False
            f = False
    return f  # Return True if all relations are in 4NF

def rel_4nf_decomposition(relations):
    # Decompose relations that violate 4NF into smaller relations
    new_tableso = {}  # Dictionary to hold new tables
    tab_already_processed = set()  # Track processed tables

    for table_name, relation in relations.items():
        prim_key = set(relation.prim_key)  # Primary key of the relation
        violate_mvd = []  # List to store MVDs that cause violations

        # Identify MVDs in the relation that violate 4NF
        for d in relation.dependency:
            lhs_set = set(d.lhs)
            rhs_set = set(d.rhs)

            if d.dependency_type == "MVD" and not is_key_superkey(relation, lhs_set):
                violate_mvd.append(d)  # Add violating MVD to the list

        if violate_mvd:
            # Process each violating MVD
            for mvd in violate_mvd:
                lhs = mvd.lhs
                rhs = mvd.rhs

                # Create new tables for each attribute on the RHS of the MVD
                for rhs_attr in rhs:
                    new_tab_name = f"{table_name}_4NF_{len(new_tableso) + 1}_Part_{rhs_attr}"
                    new_record_names = lhs + [rhs_attr]  # New record structure
                    new_tab_d = []  # Data for the new table

                    record_exist = set()  # Track existing records to avoid duplicates
                    for row in relation.data:
                        # Create new records based on the new structure
                        new_records = [row[relation.colnames.index(attr)] for attr in new_record_names]
                        if tuple(new_records) not in record_exist:
                            new_tab_d.append(new_records)  # Add to new table data
                            record_exist.add(tuple(new_records))  # Mark as existing

                    # Create the new relation for the current MVD
                    new_tables = Relation(new_record_names, new_tab_d, lhs + [rhs_attr])
                    new_tables.add_table_name(new_tab_name)  # Assign a name
                    new_tableso[new_tab_name] = new_tables  # Store the new table

            # Handle remaining fields not involved in MVD
            fields_remain = list(set(relation.colnames) - set(rhs))
            data_remain = []  # Data for remaining fields
            record_exist = set()  # Track existing records

            for row in relation.data:
                new_records = [row[relation.colnames.index(attr)] for attr in fields_remain]
                if tuple(new_records) not in record_exist:
                    data_remain.append(new_records)  # Add to remaining data
                    record_exist.add(tuple(new_records))

            # If there are remaining fields, create a new relation for them
            if len(fields_remain) > len(relation.prim_key):
                uodate_tab_name = f"{table_name}_Remaining"  # Name for remaining fields table
                update_tab = Relation(fields_remain, data_remain, relation.prim_key)
                update_tab.add_table_name(uodate_tab_name)  # Assign name
                new_tableso[uodate_tab_name] = update_tab  # Store remaining table
                tab_already_processed.add(uodate_tab_name)  # Mark as processed

        else:
            # If no MVD violations, keep the original relation
            new_tableso[table_name] = relation
            
    # Remove any duplicate relations created during decomposition
    new_tableso = remove_duplicate_relations(new_tableso)
    return new_tableso  # Return the new set of relations


In [9]:
def extract_5nf_data(relation, subset, set_compli):
    # Extract unique data for specified subsets from the relation
    idx_subsets = [relation.colnames.index(attr) for attr in subset]  # Indices for the subset attributes
    idx_comple = [relation.colnames.index(attr) for attr in set_compli]  # Indices for the complementary attributes
    
    data_a = []  # List to hold data for the subset
    data_b = []  # List to hold data for the complementary set
    data_a_exist = set()  # Set to track existing records in data_a to prevent duplicates
    data_b_exist = set()  # Set to track existing records in data_b to prevent duplicates

    for row in relation.data:
        # Create tuples for the current row's subset and complementary attributes
        row_a = tuple(row[idx] for idx in idx_subsets)
        row_b = tuple(row[idx] for idx in idx_comple)

        # Add to data_a if not already present
        if row_a not in data_a_exist:
            data_a.append(list(row_a))
            data_a_exist.add(row_a)

        # Add to data_b if not already present
        if row_b not in data_b_exist:
            data_b.append(list(row_b))
            data_b_exist.add(row_b)

    return data_a, data_b  # Return the extracted data

def verify_rel_in_5nf(relations):
    # Verify if all relations in the provided dictionary are in 5NF
    if_all_in_5nf = True  # Flag to track if all relations are in 5NF
    for n, rel in relations.items():
        # Check each relation for 5NF compliance
        if if_rel_in_5nf(rel):
            # Optionally print confirmation (commented out)
            continue
        else:
            # If any relation is not in 5NF, set the flag to False
            if_all_in_5nf = False
    return if_all_in_5nf  # Return True if all relations are in 5NF

def subset_generation(attri):
    # Generate non-trivial subsets of attributes (at least 2 elements, less than total length)
    from itertools import combinations
    return [list(c) for i in range(2, len(attri)) for c in combinations(attri, i)]

def if_rel_in_5nf(relation):
    # Check if the relation is in 5NF by examining join dependencies
    dependency_join = identify_jdepen_join(relation)  # Identify join dependencies
    return len(dependency_join) == 0  # If none exist, the relation is in 5NF

def identify_jdepen_join(relation):
    # Identify join dependencies in the relation
    jds_known = []  # List to store known join dependencies
    rel_subsets = subset_generation(relation.colnames)  # Generate subsets of attributes

    for subset in rel_subsets:
        set_compli = list(set(relation.colnames) - set(subset))  # Complementary set of attributes
        # Ensure the subsets are non-trivial
        if len(subset) >= 2 and len(set_compli) >= 2:
            # Attempt to decompose based on these attributes
            if can_be_decomposed(subset, relation):
                jds_known.append({'decomposition': (subset, set_compli)})  # Store valid decomposition

    return jds_known  # Return the identified join dependencies

def can_be_decomposed(subset, relation):
    # Check if the relation can be decomposed based on the provided subset
    set_compli = list(set(relation.colnames) - set(subset))  # Determine the complementary set
    data_a, data_b = extract_5nf_data(relation, subset, set_compli)  # Extract data for subsets

    # Simulate a natural join between the extracted data
    red_joined = simulate_natural_join(data_a, data_b, subset, set_compli)
    d_actual_set = set(tuple(row) for row in relation.data)  # Set of actual data
    red_joined_set = set(tuple(row) for row in red_joined)  # Set of joined data

    # Ensure that the data can be decomposed and prevent trivial decomposition (single columns)
    return d_actual_set == red_joined_set and len(data_a[0]) > 1 and len(data_b[0]) > 1

def extract_5nf_data(relation, subset, set_compli):
    # Redundant function, same as the first; extracting 5NF data for a relation
    idx_subsets = [relation.colnames.index(attr) for attr in subset]  # Indices for the subset
    idx_comple = [relation.colnames.index(attr) for attr in set_compli]  # Indices for the complementary set

    data_a, data_b = [], []  # Lists to hold the extracted data
    data_a_exist, data_b_exist = set(), set()  # Sets to track existing records

    for row in relation.data:
        # Create tuples for the current row's subset and complementary attributes
        row_a = tuple(row[idx] for idx in idx_subsets)
        row_b = tuple(row[idx] for idx in idx_comple)

        # Add to data_a if not already present
        if row_a not in data_a_exist:
            data_a.append(list(row_a))
            data_a_exist.add(row_a)

        # Add to data_b if not already present
        if row_b not in data_b_exist:
            data_b.append(list(row_b))
            data_b_exist.add(row_b)

    return data_a, data_b  # Return the extracted data

def simulate_natural_join(data_a, data_b, subset, set_compli):
    # Simulate a natural join between two sets of data based on overlapping attributes
    d_joined = []  # List to hold the result of the join

    for rec1 in data_a:
        for rec2 in data_b:
            # Find common attributes between subset and complementary set
            attri_of_join = list(set(subset).intersection(set_compli))
            # Check if all joining attributes match
            if all(rec1[subset.index(attr)] == rec2[set_compli.index(attr)] for attr in attri_of_join):
                # Combine rows ensuring no duplicates
                record_combine = rec1 + [rec2[set_compli.index(attr)] for attr in set_compli if attr not in attri_of_join]
                d_joined.append(record_combine)  # Add the combined record to the joined data

    return d_joined  # Return the result of the simulated natural join

def detect_dependency_join_ip(relation):
    # Detect join dependencies based on specific attributes in the relation
    col_rel1 = ['CostomerID', 'DrinkName', 'Quantity']  # First relation columns
    col_rel2 = ['OrderID', 'CostomerID', 'DrinkName']  # Second relation columns

    # Create a DataFrame from the relation data for easier manipulation
    data = pd.DataFrame(relation.data, columns=relation.colnames)

    # Get distinct records for each subset
    dta_rec1 = data[col_rel1].drop_duplicates().reset_index(drop=True)
    dta_rec2 = data[col_rel2].drop_duplicates().reset_index(drop=True)

    # Perform an inner merge on the distinct records
    d_merge = pd.merge(dta_rec1, dta_rec2, on=['CostomerID', 'DrinkName'], how='inner')
    d_actual = pd.DataFrame(relation.data, columns=relation.colnames)  # Original data as DataFrame

    d_merge_aligned = d_merge[d_actual.columns]  # Align merged data with the original columns

    # Compare the aligned merged data to the original data
    if d_merge_aligned.equals(d_actual):
        return col_rel1, col_rel2  # Return the columns if they match
    return None, None  # Return None if they do not match

def rel_5nf_decomposition_relation_ip(base_relation):
    # Decompose a base relation into 5NF if join dependencies are detected
    col_rel1, col_rel2 = detect_dependency_join_ip(base_relation)  # Identify join dependencies

    if col_rel1 and col_rel2:  # If valid columns are found
        data = pd.DataFrame(base_relation.data, columns=base_relation.colnames)  # Create DataFrame

        dta_rec1 = data[col_rel1].drop_duplicates().values.tolist()  # Extract unique records for first relation
        dta_rec2 = data[col_rel2].drop_duplicates().values.tolist()  # Extract unique records for second relation

        # Define primary keys for the decomposed relations
        r1_prim_key = ['CostomerID', 'DrinkName', 'Quantity']
        r2_prim_key = ['OrderID', 'CostomerID', 'DrinkName']

        # Create Relation instances for R1 and R2
        rec1_rel = Relation(col_rel1, dta_rec1, prim_key=r1_prim_key)
        rec2_rel = Relation(col_rel2, dta_rec2, prim_key=r2_prim_key)

        # Assign names to the new relations
        rec1_rel.add_table_name("Table1")
        rec2_rel.add_table_name("Table2")

        return rec1_rel, rec2_rel  # Return the new relations
    return None, None  # Return None if no join dependencies found

def rel_5nf_decomposition(relations):
    # Decompose a list of relations into 5NF if necessary
    rel_decomp = []  # List to hold decomposed relations

    for relation in relations:
        print(f"Checking relation: {relation.name}")  # Print current relation being checked

        # Step 1: Check if the relation is already in 5NF
        if if_rel_in_5nf(relation):
            print(f"Relation '{relation.name}' is already in 5NF.")
            rel_decomp.append(relation)  # Add the relation if it is in 5NF
            continue

        # Step 2: Detect join dependency for potential 5NF violations
        dependency_join = identify_jdepen_join(relation)

        # If there are no valid join dependencies, it is already in 5NF
        if not dependency_join:
            rel_decomp.append(relation)
            continue

        # Step 3: Decompose based on the detected join dependency
        for jd in dependency_join:
            subset1, subset2 = jd['decomposition']  # Get the subsets for decomposition

            # Extract data for each subset
            data_a, data_b = extract_5nf_data(relation, subset1, subset2)

            # Create new Relation instances for the decomposed parts
            rel1 = Relation(subset1, data_a, prim_key=subset1)
            rel1_name = f"{relation.name}_PartA"  # Name for the first decomposed part
            rel1.add_table_name(rel1_name)
            rel_decomp.append(rel1)  # Add first relation to decomposed list

            rel2 = Relation(subset2, data_b, prim_key=subset2)
            rel2_name = f"{relation.name}_PartB"  # Name for the second decomposed part
            rel2.add_table_name(rel2_name)
            rel_decomp.append(rel2)  # Add second relation to decomposed list

            print(f"Decomposed '{relation.name}' into '{rel1_name}' and '{rel2_name}'.")

    return rel_decomp  # Return the list of decomposed relations


In [10]:
def reading_dependency(filename):
    # Read functional and multi-valued dependencies from a file and organize them by relation
    dependency = {}  # Dictionary to hold dependencies for each relation
    current_relation = None  # Track the currently processed relation

    with open(filename, 'r') as file:
        for line in file:
            line = line.strip()  # Remove any leading/trailing whitespace
            
            # Detect a new relation (indicated by a line ending with a colon)
            if line.endswith(":"):
                current_relation = line[:-1]  # Set current_relation to the line without the colon
                dependency[current_relation] = []  # Initialize an empty list for dependencies
            elif line:
                # Append functional or multi-valued dependency to the current relation's list
                dependency[current_relation].append(line)

    return dependency  # Return the organized dependencies

def validatinf_dependency(relations, dependency):
    # Validate and add dependencies to the given relations based on the dependency dictionary
    for relation_name, fds in dependency.items():
        relation = relations.get(relation_name)  # Retrieve the relation by its name
        if not relation:
            print(f"Relation '{relation_name}' not found.")  # Notify if the relation is not present
            continue

        # Uncomment the following lines to display dependencies (currently commented out)
        # print(f"\ndependency for '{relation_name}':")
        # for fd in fds:
        #     print(f"  - {fd}")

        # Uncomment for user validation (currently commented out)
        # user_input = input(f"Do you confirm all dependency for '{relation_name}'? (yes/no): ").strip().lower()

        # if user_input == "yes":
        #     # Add all dependencies to the relation
        for fd in fds:
            # Create a Dependency object for each functional/multi-valued dependency
            dependency = Dependency(fd, [relation.prim_key])  # Assuming prim_key is the only candidate key for simplicity
            relation.dependency_adding(dependency)  # Add the dependency to the relation
        #     print(f"dependency added to relation '{relation_name}'.")
        # else:
        #     print(f"Skipping dependency for '{relation_name}'.")

        # Uncomment the following lines to allow user to add additional dependencies (currently commented out)
        # while user_input != "done":
        #     user_input = input(f"Enter additional dependency for '{relation_name}' in 'LHS -> RHS' or 'LHS -->> RHS' format (or 'done' to finish): ").strip()
        #     if user_input != "done":
        #         dependency = Dependency(user_input, [relation.prim_key])
        #         relation.dependency_adding(dependency)
        #         print(f"Added dependency '{user_input}' to relation '{relation_name}'.")

    return relations  # Return the updated relations with added dependencies


In [11]:
def detection_of_max_nf(if_in_1nf_relation):
   
    
    # Check if the relations are in 1NF
    if if_in_1nf_relation:
        # Check if all relations are in 2NF
        if check_all_relations_2nf(dictionaryformat(relations)):
             # If in 2NF, check if they are also in 3NF
            if relations_in_3nf(dictionaryformat(relations_2nf)):
                # If in 3NF, check if they are in BCNF (stricter version of 3NF)
                if relation_is_in_bcnf(dictionaryformat(relations_3nf)):
                    
                    if verify_relations_in_4nf(dictionaryformat(relations_bcnf)):
                        
                        if verify_rel_in_5nf(dictionaryformat(relations_4nf)):
                            return "Highest Normal Form: 5NF"
                        else:
                            return "Highest Normal Form: 4NF"        
                    else:
                        return "Highest Normal Form: BCNF"  # If in BCNF, it's the highest
                else:
                    return "Highest normal form: 3NF"  # If not BCNF, highest is 3NF
            else:
                return "Highest normal form: 2NF"  # If not in 3NF, highest is 2NF
           
        else:
            return "Highest normal form: 1NF"  # If not in 2NF, highest is 1NF
    
    # If the relation doesn't meet 1NF requirements, return 0NF (un-normalized)
    return "Highest normal form: 0NF"


In [12]:
def dictionaryformat(rltn):
    # Convert a Relation object into a dictionary format for easier access
    if isinstance(rltn, Relation):
        return {rltn.name: rltn}  # Return a dictionary with the relation name as the key
    return rltn  # If not a Relation, return the input as is

def main():
    # Read data from an Excel file
    input_file = 'inputtable.xlsx'
    dataf = pd.read_excel(input_file)  # Load the data into a DataFrame

    fd_filename = 'inputfds.txt'  # File containing functional and multi-valued dependencies

    colnames = list(dataf.columns)  # Get column names from the DataFrame
    data = dataf.values.tolist()  # Convert DataFrame to a list of lists for data

    # Get primary keys from user input
    prim_key = input("Enter the primary keys, separated by commas: ").split(',')
    prim_key = [key.strip() for key in prim_key]  # Strip whitespace from keys

    # Create a Relation object with the data and primary key
    relation = Relation(colnames, data, prim_key)
    relation.add_table_name("InputRelation")  # Assign a name to the relation

    # Check if the relation is in 1NF and normalize if needed
    if_in_1nf_relation, not_1nf_attr = is_relation_1nf(relation)

    # Ask user if they want to detect the highest normal form
    highest_nf_of_ip = int(input("Detect Highest Normal form of table: Yes(1), No(0): "))
    if highest_nf_of_ip == 1:
        print(detection_of_max_nf(if_in_1nf_relation))  # Print the highest normal form if requested
    else:
        pass

    # Present normalization options to the user
    print("Select the highest level of normalization you want to achieve:")
    print("Select 1 - Normalisation up to 1NF")
    print("Select 2 - Normalisation up to 2NF")
    print("Select 3 - Normalisation up to 3NF")
    print("Select 4 - Normalisation up to BCNF")
    print("Select 5 - Normalisation up to 4NF")
    print("Select 6 - Normalisation up to 5NF")

    option = int(input("Enter your option (1-6): "))  # Get the normalization option from the user

    # If the relation is already in 1NF, validate dependencies
    if if_in_1nf_relation:
        print("All Tables are in 1NF")
        dependency = reading_dependency(fd_filename)  # Read dependencies from file
        rltns = validatinf_dependency(dictionaryformat(relation), dependency)  # Validate and add dependencies

    else:
        # Normalize to 1NF if not already in 1NF
        tables_dictionary_normalised = normalize_1nf_relation(relation, not_1nf_attr)
        dependency = reading_dependency(fd_filename)
        rltns = validatinf_dependency(tables_dictionary_normalised, dependency)  # Validate dependencies
        print("AFTER 1NF NORMALIZATION")
        for r in tables_dictionary_normalised.values():
            r.show()  # Display normalized tables
        for r in tables_dictionary_normalised.values():
            print(r.table_formation())  # Show table formation

    if option == 1:
        return  # Exit if only normalization to 1NF is required

    # Decompose to 2NF
    print("-------------Tables of 2NF-----------------")
    if check_all_relations_2nf(dictionaryformat(rltns)):
        print("TABLES ARE IN 2NF")
        relations_2nf = rltns  # If already in 2NF, keep the relations as is
    else:
        # Normalize to 2NF
        relations_2nf = decompose_to_2nf(dictionaryformat(rltns))
        print("AFTER 2NF NORMALIZATION")
        for rel in relations_2nf.values():
            rel.show()  # Show 2NF tables
        for rel in relations_2nf.values():
            print(rel.table_formation())

    if option == 2:
        return  # Exit if only normalization to 2NF is required

    # Decompose to 3NF
    print("-----------Tables of 3NF----------")
    if relations_in_3nf(dictionaryformat(relations_2nf)):
        print("TABLES ARE IN 3NF")
        relations_3nf = relations_2nf  # Keep the relations as is if in 3NF
    else:
        # Normalize to 3NF
        relations_3nf = decompose_to_3nf(dictionaryformat(relations_2nf))
        print("AFTER 3NF NORMALIZATION")
        for rel in relations_3nf.values():
            rel.show()  # Show 3NF tables
        for rel in relations_3nf.values():
            print(rel.table_formation())
        print("-----------------------------------------------------------------------------------------------------------------")

    if option == 3:
        return  # Exit if only normalization to 3NF is required

    # Decompose to BCNF
    print("----------Tables of BCNF--------------")
    if relation_is_in_bcnf(dictionaryformat(relations_3nf)):
        print("TABLES ARE IN BCNF")
        relations_bcnf = relations_3nf  # Keep the relations if already in BCNF
    else:
        # Normalize to BCNF
        relations_bcnf = bcnf_decomposition(dictionaryformat(relations_3nf))
        print("AFTER BCNF NORMALIZATION")
        for rel in relations_bcnf.values():
            rel.show()  # Show BCNF tables
        for rel in relations_bcnf.values():
            print(rel.table_formation())

    if option == 4:
        return  # Exit if only normalization to BCNF is required

    # Decompose to 4NF
    print("-----------Tables of 4NF------------")
    if verify_relations_in_4nf(dictionaryformat(relations_bcnf)):
        print("TABLES ARE IN 4NF")
        relations_4nf = relations_bcnf  # Keep the relations if already in 4NF
    else:
        # Normalize to 4NF
        relations_4nf = rel_4nf_decomposition(dictionaryformat(relations_bcnf))
        print("AFTER 4NF NORMALIZATION")
        for rel in relations_4nf.values():
            rel.show()  # Show 4NF tables
        for rel in relations_4nf.values():
            print(rel.table_formation())
        print("-----------------------------------------------------------------------------------------------------------------")

    if option == 5:
        return  # Exit if only normalization to 4NF is required

    # Decompose to 5NF
    print("-------------5NF----------------")
    if verify_rel_in_5nf(dictionaryformat(relations_4nf)):
        print("TABLES ARE IN 5NF")
    else:
        # Normalize to 5NF
        relations_5nf = rel_5nf_decomposition(list(dictionaryformat(relations_4nf).values()))
        print("AFTER 5NF NORMALIZATION")
        for rel in relations_5nf:
            rel.show()  # Show 5NF tables
        for rel in relations_5nf:
            print(rel.table_formation())

# if __name__ == "__main__":
main()  # Call the main function to execute the program


Enter the primary keys, separated by commas:  OrderID,FoodID,DrinkID
Detect Highest Normal form of table: Yes(1), No(0):  1


Highest normal form: 0NF
Select the highest level of normalization you want to achieve:
Select 1 - Normalisation up to 1NF
Select 2 - Normalisation up to 2NF
Select 3 - Normalisation up to 3NF
Select 4 - Normalisation up to BCNF
Select 5 - Normalisation up to 4NF
Select 6 - Normalisation up to 5NF


Enter your option (1-6):  6


AFTER 1NF NORMALIZATION
BaseRelation
+---------+---------------------+-----------+----------------+---------------+------------+--------------+---------+--------------------------+-----------+---------------+------+--------+------------------+--------------+
| OrderID |        Date         | TotalCost | TotalDrinkCost | TotalFoodCost | CustomerID | CustomerName | DrinkID |        DrinkName         | DrinkSize | DrinkQuantity | Milk | FoodID |     FoodName     | FoodQuantity |
+---------+---------------------+-----------+----------------+---------------+------------+--------------+---------+--------------------------+-----------+---------------+------+--------+------------------+--------------+
|  1001   | 2024-06-30 00:00:00 |   7.25    |      7.25      |      0.0      |     1      | Alice Brown  |    1    |       Caffe Latte        |  Grande   |       1       |  ND  |   0    |       nan        |      0       |
|  1002   | 2026-06-30 00:00:00 |   9.98    |      5.99      |     3.99    

In [13]:
# Specify the input file containing data for 5NF decomposition
input_file = '5nfdata.xlsx'

# Read the Excel file into a pandas DataFrame
dataf = pd.read_excel(input_file)

# Extract column names and data from the DataFrame
colnames = list(dataf.columns)  # Get the list of column names
data = dataf.values.tolist()  # Convert DataFrame to a list of lists for data

# Define the primary keys for the relation
prim_key = ['OrderID', 'Quantity', 'DrinkName', 'CostomerID']

# Create an instance of the Relation class with the column names, data, and primary keys
relation = Relation(colnames, data, prim_key)

# Assign a name to the relation for identification
relation.add_table_name("Orders")

# Attempt to decompose the relation into 5NF relations, if applicable
rel1, rel2 = rel_5nf_decomposition_relation_ip(relation)

# Check if the decomposition resulted in valid relations
if rel1 and rel2:
    print("Decomposition is done! Tables are now in 5NF:")
    rel1.show()  # Display the first decomposed relation
    rel2.show()  # Display the second decomposed relation
else:
    print("No valid join dependency. Relations are in 5NF.")  # Indicate that no decomposition was needed


Decomposition is done! Tables are now in 5NF:
Table1
+------------+-----------+----------+
| CostomerID | DrinkName | Quantity |
+------------+-----------+----------+
|     1      |  Coffee   |  Large   |
|     1      |  Coffee   |  Medium  |
|     2      |   Coke    |  Large   |
|     2      |   Coke    |  Small   |
|     3      |  Sprite   |  Medium  |
|     3      |  Sprite   |  Small   |
+------------+-----------+----------+
Primary Key: ['CostomerID', 'DrinkName', 'Quantity']
Functional dependencies for tables are:
Table2
+---------+------------+-----------+
| OrderID | CostomerID | DrinkName |
+---------+------------+-----------+
|  1001   |     1      |  Coffee   |
|  1002   |     2      |   Coke    |
|  1003   |     3      |  Sprite   |
+---------+------------+-----------+
Primary Key: ['OrderID', 'CostomerID', 'DrinkName']
Functional dependencies for tables are:
